# 第二十章 迭代和解析

列表解析: 用于把任意一个表达式而不是一个函数应用于一个迭代对象中的元素, 比 map 和 filter 更加通用

__列表解析与 map__

In [1]:
res = list(map(ord,'spam'))
res

[115, 112, 97, 109]

列表解析用于在一个序列上应用一个任意的表达式,将其结果收集到一个新的列表中并返回

In [ ]:
res = [ord(x) for x in 'spam']
res

[115, 112, 97, 109]

In [ ]:
list(map((lambda x: x**2), range(10)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

__列表解析增加循环和嵌套__

In [ ]:
[x for x in range(10) if x % 2 ==0]

[0, 2, 4, 6, 8]

In [ ]:
list(filter((lambda x: x % 2 == 0), range(10)))

[0, 2, 4, 6, 8]

In [ ]:
res = [x+y for x in range(3) if x % 2==0 for y in range(3) if y%2==0]
res

[0, 2, 2, 4]

__列表解析和矩阵__

In [ ]:
M = [
    [1,2,3],
    [4,5,6],
    [7,8,9]
]
N = [
    [2,2,2],
    [3,3,3],
    [4,4,4],
]

In [ ]:
M[1]

[4, 5, 6]

In [ ]:
M[1][2]

6

In [ ]:
[row[0] for row in M]

[1, 4, 7]

### 理解列表解析

对于现在运行的 Python 下的测试, map 调用比等效的 for 循环快两倍,列表解析比 map 循环更加快. 主要是因为 map 和 列表是在解释器中以 C 语言的速度来运行的

1. for 循环可以让逻辑变得更加清晰,建议使用
2. map 和列表解析也是比较容易理解的,并且能够改善应用运行的速度

In [ ]:
open('data.x').readlines()

['aa\n', 'bbhello \n', 'world \n']

In [ ]:
[line.strip() for line in open('data.x')]

['aa', 'bbhello', 'world']

In [ ]:
[line.rstrip() for line in open('data.x')]

['aa', 'bbhello', 'world']

In [ ]:
list(map((lambda line: line.strip()), open('data.x')))

['aa', 'bbhello', 'world']

## 生成器

Python 对于延迟提供了更多的支持 --- 它提供了工具在需要的时候才产生结果,而不是立即产生结果. 可以通过两种语言结构延迟结果创建

* 生成器函数: 编写为常规的 def 语句,但是使用 yield 语句一次返回一个结果,在每个结果之间挂起和继续他们的状态
* 生成器表达式: 返回按照需要产生结果的一个对象, 而不是构建一个结果列表

### 生成器函数 yield VS return
* 生成器函数: 返回一个值并且从随后退出的地方继续的函数. 生成器函数是随着时间产生的一个序列
* 创建时自动实现迭代器协议

__状态挂起__
* 生成器函数自动在生成值的时刻挂起并继续函数的执行. 通常用于提起计算一系列值以及在类中手动保存和恢复状态都很有用
* 生成器函数在挂起时保存的状态包含他们的整个本地作用域, 当函数恢复时, 他们的本地变量保存了信息并且使得其可用
* yield 语句挂起该函数并且像调用者返回一个值,但是保留足够的状态使得函数能够从他们离开的地方继续. 
* 需要继续时, 函数在上一个 yield 返回后立即继续执行

__迭代协议整合__

对于生成器协议的支持:
1. 函数需要包含一条 yield 语句, 进行调用时, 他们返回一个迭代器对象, 该对象支持用一个名为__next__ 的自动创建的方法来继续执行的接口
2. 生成器的 return 位于 def 语句块的末尾, 用于终止值的生成

__生成器函数__

In [1]:
def gen_squares(N):
    for i in range(N):
        yield i**2
    return

In [3]:
for i in gen_squares(5):
    print(i, end=':')

0:1:4:9:16:

In [4]:
x = gen_squares(4)

In [5]:
x

<generator object gen_squares at 0x1038acf90>

In [6]:
next(x)

0

In [7]:
next(x)

1

生成器在内存使用和性能方面都更好. 允许函数避免临时在做所有的工作. 当结果的列表很大或者在处理每一个结果都需要很多的时间时, 非常有用

## 生成器表达式
* 生成器表达式就像一般的列表解析一样, 他们是在圆括号中,而不是方括号中的
* 生成器表达式不在内存中构建结果,而是返回一个生成器对象, 该对象支持迭代器协议


In [9]:
G = (x ** 2 for x in range(4))

In [10]:
next(G)

0

In [11]:
next(G)

1

In [12]:
next(G)

4

生成器表达式可以对于内存空间进行优化,但是不一定对于性能能够进行优化

In [15]:
G = (c*4 for c in 'spam')
list(G)

['ssss', 'pppp', 'aaaa', 'mmmm']

In [16]:
def time_four(s):
    for c in s:
        yield c * 4

In [17]:
list(time_four('spam'))

['ssss', 'pppp', 'aaaa', 'mmmm']

生成器表达式和生成器函数自身都是迭代器,但是只支持一次活跃迭代. 一旦任何迭代器运行到完成, 所有的迭代器都将耗尽

### 迭代工具模拟 zip 和 map


模拟 map 函数

In [18]:
def my_map(func, *seqs):
    res = []
    for args in zip(*seqs):
        res.append(func(*args))
    
    return res

In [19]:
print(my_map(abs, [0,-1,2,-3]))

[0, 1, 2, 3]


In [20]:
print(my_map(pow, [1,2,3,4],[3,4,5,6,]))

[1, 16, 243, 4096]


使用列表解析可以更简单的实现上述功能

In [21]:
def my_map(func, *seqs):

    return (func(*args) for args in zip(*seqs))

In [24]:
print(
    my_map(
        pow,
        [1, 2, 3, 4],
        [
            3,
            4,
            5,
            6,
        ],
    )
)

[1, 16, 243, 4096]


实现自己的 zip 和 map

不带有默认版本的 zip

In [2]:
def my_zip(*seqs, pad=None):
    seqs = [list(item) for item in seqs]
    res = []
    while any(seqs):
        res.append(tuple((seq.pop(0) if seq else pad) for seq in seqs))

    return res

In [3]:
s1 = "abc"
s2 = "xyz123"

print(my_zip(s1, s2,pad=99))

[('a', 'x'), ('b', 'y'), ('c', 'z'), (99, '1'), (99, '2'), (99, '3')]


生成器版本的 zip

In [4]:
def my_zip(*seqs, pad=None):
    seqs = [list(item) for item in seqs]
    while any(seqs):
        yield tuple(seq.pop(0) if seq else pad for seq in seqs)

In [6]:
s1 = "abc"
s2 = "xyz123"

print(list(my_zip(s1, s2, pad=99)))

[('a', 'x'), ('b', 'y'), ('c', 'z'), (99, '1'), (99, '2'), (99, '3')]


In [35]:
import time
import sys
reps = 10000
repslist= range(reps)

def timer(func, *args,**kwargs):
    start = time.time()
    for _ in repslist:
        ret = func(*args,**kwargs)
    elapsed = time.time() - start

    return elapsed,ret

In [37]:
def for_loop():
    res = []
    for x in repslist:
        res.append(abs(x))
    
    return res

def list_comp():
    return [abs(x) for x in repslist]

def map_call():
    
    return list(map(abs, repslist))

def gen_expr():
    return list(abs(x) for x in repslist)

def gen_func():
    def gen():
        for x in repslist:
            yield abs(x)
    
    return list(gen())    


开始记录时间

In [38]:
print(sys.version)

3.10.7 (v3.10.7:6cc6b13308, Sep  5 2022, 14:02:52) [Clang 13.0.0 (clang-1300.0.29.30)]


In [39]:
for test in [for_loop, list_comp,map_call, gen_expr,gen_func]:
    elapsed,result = timer(test)
    print('-'*33)
    print("%-9s: %.5f => [%s...%s]" % (test.__name__, elapsed, result[0],result[-1]))

---------------------------------
for_loop : 4.50421 => [0...9999]
---------------------------------
list_comp: 2.83421 => [0...9999]
---------------------------------
map_call : 1.60770 => [0...9999]
---------------------------------
gen_expr : 4.03837 => [0...9999]
---------------------------------
gen_func : 4.01984 => [0...9999]


# 第二十一章: 模块

## import 是如何工作的

Python 中的 import 并非是把一个文件插入到另一个文件中. 导入其实是运行时的运算, 程序第一次导入指定文件时,会执行三个步骤:
1. 找到模块文件
2. 编译成位码
3. 执行模块的代码来创建其所定义的对象

1. __搜索__: Python 必须查找 import 语句所引用的模块文件. Python 使用了标准模块搜索路径来查找 import 语句所对应的模块文件
2. __编译__: 如果有需要会编译为字节码, 如果存在字节码,直接加载字节码. 如果发现字节码的文件比源代码的文件就,就会重新编译字节码
3. __运行__: 依次执行文件中所有语句的字节码文件

### 搜索
1. 程序的主目录. 找到即停止, 先搜索先使用
2. PYTHONPATH 目录(如果进行了设置)
3. 任何.pth 文件的内容 (如果存在该文件)
4. 标准库链接目录

已经导入的模块查看

In [43]:
list(sys.modules.keys())

['sys',
 'builtins',
 '_frozen_importlib',
 '_imp',
 '_thread',
 '_warnings',
 '_weakref',
 '_io',
 'marshal',
 'posix',
 '_frozen_importlib_external',
 'time',
 'zipimport',
 '_codecs',
 'codecs',
 'encodings.aliases',
 'encodings',
 'encodings.utf_8',
 '_signal',
 '_abc',
 'abc',
 'io',
 '__main__',
 '_stat',
 'stat',
 '_collections_abc',
 'genericpath',
 'posixpath',
 'os.path',
 'os',
 '_sitebuiltins',
 '__future__',
 'itertools',
 'keyword',
 '_operator',
 'operator',
 'reprlib',
 '_collections',
 'collections',
 'types',
 '_functools',
 'functools',
 'contextlib',
 '_virtualenv',
 '_distutils_hack',
 'site',
 'importlib._bootstrap',
 'importlib._bootstrap_external',
 'warnings',
 'importlib',
 'importlib.machinery',
 'importlib._abc',
 'importlib.util',
 'runpy',
 'enum',
 '_sre',
 'sre_constants',
 'sre_parse',
 'sre_compile',
 '_locale',
 'copyreg',
 're',
 'fnmatch',
 'ntpath',
 'errno',
 'urllib',
 'urllib.parse',
 'pathlib',
 'collections.abc',
 'typing.io',
 'typing.re',
 '

#### 配置搜索路径


查看平台配置的模块搜索路径; python 会从左到右自动搜索该文件的路径

In [1]:
import sys
print(sys.path)

['/Users/lixun1/PycharmProjects/DatascientificProject/notebooks', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/lixun1/PycharmProjects/DatascientificProject/venv/lib/python3.10/site-packages']


默认的 sys 目录添加:
1. 顶级文件的主目录
2. 任何 PYTHONPATH 目录
3. 已经创建的包含 .pth 文件路径的内容
4. 标准库目录

1. 通过修改 sys.path 进行导入路径的临时性修改
2. 通过 PYTHONPATH 和 .pth 用于提供路径的持久化修改

模块的处理函数:
1. import: 使用一个变量名引用整个模块变量, 需要通过模块名来获取对象的属性
2. from: 将变量名复制到该作用域, 建议直接使用该变量名
3. from *: 将所有变量名引用到该作用域
4. impl.reload(): 不终止程序的情况下,提供了一种重新载入模块文件代码的方法

导入只有在第一次import 或 from 时,载入并且执行,并且只执行一次

import 和 from 是可执行的语句,而不是编译期间的声明, 可以嵌套在 if 测试中,出现在函数 def 之中
1. import 将整个模块的对象赋值给一个变量名
2. from 用于将一个或者多个变量名赋值给另一个模块中同名的对象

reload:
1. reload 是 Python 中的内置函数, 而不是语句
2. 传给 reload 的是已经存在的模块对象, 而不是变量名
3. 

__init__.py: 包导入语句的路径中的每个目录内,都必须有 __init__.py 文件,否则包导入会失败. 
1. dir1 和 dir2 中必须有一个 __init__py 文件
2. dir0 是容器,不需要 __init__.py

1. 包的初始化: Python 首次导入某个目录时, 会自动执行该目录下 __init__.py 文件中的所有程序代码. 包含创建数据文件 连接数据库
2. 模块命名空间的初始化: 在包导入的模型中, 目录内的目录路径, 在导入后会变成真实的嵌套对象路径. 
3. from 语句的行为: 可以在 __init__.py 文件内使用  __all__ 列表来定义

# 第二十四章 在模块中隐藏数据

* 在变量名前防止 _x 可以用于防止客户端使用 from * 语句导入模块时, 把其中的那些变量名复制出去
* 在模块顶层把变量名的字符串列表赋值给变量 __all__, 用于达到类似 _x 命名惯例的隐藏效果
  __all__ = ['Error', 'encode','decode'] 用于指出需要复制的变量名

* __name__ : 如果文件是以顶层程序文件运行的话, 在启动时, __name__ 就会被设置为字符串 __main__ 
* 如果文件被导入, __name__ 就会改成客户端所了解的模块名
* 模块可以检测自己的 __name__, 来确定他是在执行还是在导入
* __name__ 可以用来进行最简单的单元测试

In [5]:
def minmax(test, *args):
    res = args[0]
    for arg in args[1:]:
        if test(arg,res):
            res = arg

    return res

def lessthan(x,y): return x < y

def grtrthan(x,y): return x > y


if __name__=='__main__':
    print(minmax(lessthan, 4,3,2,1,4,5,6,3))
    print(minmax(grtrthan, 4,3,2,1,4,5,6,3))    
    

1
6


## 修改模块的搜索路径

sys.path 在程序启动时进行初始化，并在此之后，可以任意对其元素进行删除，附加和重设.

In [7]:
import sys
sys.path

['/Users/lixun1/PycharmProjects/DatascientificProject/notebooks',
 '/opt/anaconda3/lib/python311.zip',
 '/opt/anaconda3/lib/python3.11',
 '/opt/anaconda3/lib/python3.11/lib-dynload',
 '',
 '/opt/anaconda3/lib/python3.11/site-packages',
 '/opt/anaconda3/lib/python3.11/site-packages/aeosa']

In [8]:
sys.path.append('')

## 模块是对象： 元程序
模块通过内置属性显示了他们大多数的有趣的属性，因此可以很容易的编写程序来管理其他程序。这类程序称为元程序，在其他系统之上工作，也称为内省。程序可以看见可处理对象的内部

* M.name
* M.__dict__['name']
* sys.modules['M'].name
* getattr(M,'name')

# 第二十五章: OOP 宏伟蓝图

* OOP 提供了一种不同寻常的而且往往是更有效的检查程序的方式,利用这种设计方法,分解代码,降低代码的冗余度,通过定制现有的代码来编写新的程序,而不是在原处修改
* 类的设计可以用来创建和管理新的对象
* 类的两个作用:
* * 继承: 继承父类的属性,并且扩展子类的属性
* * 组合: 通过组合其他对象实例,来完成相应的指令,定义自己的行为以及类关系
* * 类是对象的工厂: 类基本上就是产生对象的工厂, 每次调用一个类, 就会产生一个有独立命名空间的新对象. 每个通过类产生的对象都可以读取类的属性,并获得自己的命名空间来存储数据
* * 通过继承进行定制: 支持在类的外部重新定义其属性,从而扩充该类.
* * 运算符重载: 类可以定义对象来响应在内置类型上的几种运算. 切片 级联和索引等运算. 通过类钩子可以用来试下任何内置类型的运算


## 概览 OOP



* 像函数那样调用类会创建新的实例对象
* 每个实例对象继承类的属性并且获得了自己命名空间
* 在方法内对 self 属性做赋值运算会产生每个实例自己的属性

In [1]:
class FirstClass:
    def setdata(self, value):
        self.data = value
    def display(self):
        print(self.data)


In [2]:
x = FirstClass()
y = FirstClass()

In [3]:
x.setdata("King Arthur")
y.setdata(3.21156)

In [4]:
x.display()

King Arthur


In [5]:
y.display()

3.21156


In [6]:
x.data = "New Value"
x.display()

New Value


In [7]:
x.anothername = "name"

In [8]:
x.anothername

'name'

* 超类列在了类开头的括号中
* 类从其超类中继承属性
* 实例会继承所有可读取类的属性
* 每个 object.attribute 都会开启新的独立的搜索
* 逻辑的修改是通过创建子类，而不是修改超类

In [9]:
class SecondClass(FirstClass):
    def display(self):
        print('Current value="%s"' % self.data)

In [10]:
z = SecondClass()
z.setdata(42)
z.display()

Current value="42"


* 类可以截获 Python 运算符
* 以 __x__ 双下换线命名的方法是特殊的钩子。Python 运算符重载的实现是提供特殊命名的方法来拦截运算。Python 语言替每种运算和特殊命名的方法之间，定义了固定不变的映射关系
* 当实例出现内置运算时，这些方法会自动调用
* 类可覆盖多数内置类型运算
* 运算符覆盖方法没有默认值
* 运算符可以让类与 Python 的对象模型相互集成

In [13]:
class ThirdClass(SecondClass):
    def __init__(self,value):
        self.data = value
    def __add__(self, other):
        return ThirdClass(self.data + other)
    def __str__(self):
        return '[ThirdClass: %s]' % self.data
    def mul(self,other):
        self.data *= other

In [14]:
a = ThirdClass('abc')
a.display()

Current value="abc"


In [15]:
print(a)

[ThirdClass: abc]


In [16]:
b = a + 'xyz'

In [17]:
print(b)

[ThirdClass: abcxyz]


In [18]:
a.mul(3)

In [19]:
print(a)

[ThirdClass: abcabcabc]


最简单的类

In [20]:
class rec:pass

In [22]:
rec.name = 'bob'
rec.age=24

In [23]:
print(rec.name)
print(rec.age)

bob
24


In [24]:
x = rec()
y = rec()

In [25]:
x.name,y.name

('bob', 'bob')

如果一个列没有实例化，为其动态添加的属性是存储在类空间的

In [26]:
x.name='Sun'

In [28]:
x.name,y.name,rec.name

('Sun', 'bob', 'bob')

命名空间对象的属性通常都是以字典的形式实现的，而类继承树只是连接到其他字典的字典而已

In [30]:
rec.__dict__.keys()

dict_keys(['__module__', '__dict__', '__weakref__', '__doc__', 'name', 'age'])

In [31]:
x.__dict__.keys()

dict_keys(['name'])

In [32]:
y.__dict__.keys()

dict_keys([])

每个实例都会连接到其类以便于进行继承，可以通过 ____class____ 进行查看

In [33]:
x.__class__

__main__.rec

类有一个__bases__属性，是其超类的元组

In [34]:
rec.__bases__

(object,)

In [36]:
x.__class__

__main__.rec

Python 的类和实例只是命名空间对象，属性是通过赋值语句动态建立
及时是方法也可以完全独立的在任意对象的外部创建

In [37]:
def uppperName(self):
    return self.name.upper()

In [38]:
uppperName(x)

'SUN'

将函数赋值给类，作为属性

In [39]:
rec.method = uppperName

In [41]:
x.method()

'SUN'

In [42]:
y.method()

'BOB'

In [44]:
rec.method(x)

'SUN'

#### 类与字典的关系 

In [45]:
rec = {}
rec['name'] = 'mel'
rec['age'] = 45
rec['job'] = 'trainer/writer'

In [46]:
print(rec['name'])

mel


通过类来记录参数的代价要比字典等价形式少很多。 通过空类来产生命名空间对象

In [47]:
class rec:pass

In [48]:
rec.name = 'mel'
rec.age = 45
rec.job = 'trainer/writer'

In [49]:
print(rec.age)

45


# 第二十七章

OOP 中的核心概念：
1. 创建实例：填充实例属性
2. 定义行为方法：在类方法中封装逻辑
3. 运算符重载：为打印行为提供内置的操作行为
4. 定制行为：重新定义子类中的方法使其特殊化
5. 定制构造函数：为超类步骤添加初始化逻辑

委托：委托是一种常用代码模式的代表，是一种复合的结构，管理一个包装的对象并且吧方法调用传递给他

# 第二十八章： 类编写细节


把简单的非函数的对象赋值给类属性，就会产生数据属性，并且由实例共享

In [1]:
class SharedData:
    spam = 42


In [3]:
x=SharedData()
y=SharedData()
x.spam,y.spam

(42, 42)

可以通过类名修改，通过实例或者类引用

In [4]:
SharedData.spam=99
x.spam,y.spam

(99, 99)

通过实例进行修改不会影响所有共享变量的值，而是创建对应的实例变量

In [6]:
x.spam=42
x.spam,y.spam,SharedData.spam

(42, 99, 99)

* 对于实例的属性进行赋值运算会在该实例内创建或者修改变量名，而不是在共享的类中
* 继承搜索只会发生在属性引用时，而不是在赋值运算时发生：对对象属性的赋值总是会修改该对象


In [14]:
class MixedNames:
    data = 'spam'
    def __init__(self,value):
        self.data = value
    def display(self):
        print(self.data,MixedNames.data)

通过构造函数进行修改

In [15]:
x = MixedNames(1)
y = MixedNames(2)

In [18]:
x.display();y.display()

1 spam
2 spam


将属性存储在不同的对象内，可以决定其可见范围；附加在类上时，变量名是共享的；附加在实例上时，变量名是属于每个实例的数据，非共享数据

## 方法
* Python 会自动将实例方法的调用对应到类方法函数；
* class 通过Python 继承搜索流程找出方法名称所在之处
* 通过 self 提供的钩子，返回调用的主体，本质上就是实例对象
* 在 class 语句的作用域中赋值的对象，就会变成类对象的属性，然后由这个类创建的每个实例继承

In [19]:
class NextClass:
    def printer(self,text):
        self.message = text
        print(self.message)

In [20]:
x = NextClass()
x.printer('instance call')

instance call


In [21]:
x.message

'instance call'

方法只能够通过实例或类本身两种两种方法中的任意一种执行; 通过累的形式进行传递有相同的效果，只需要在类形式中传递了相同的实例对象

In [25]:
NextClass.printer(x,'class call')

class call


In [26]:
x.message

'class call'

## 调用超类构造函数

方法一般是通过实例调用的，但是对于构造函数之类的属性__init__ 都是通过继承进行查找的，Python 找出并且只调用一个 __init__. 如果要保证子类的构造函数执行超类的构造逻辑，一般需要通过类明确的调用超类的__init__ 方法

In [37]:
class Super:
    def __init__(self,x):
        self.x = x
class Sub(Super):
    def __init__(self,x,y):
        # Super.__init__(self,x)
        super().__init__(x)
        self.y = y 
    

In [38]:
I = Sub(1,2)

In [39]:
I.x

1

In [40]:
I.y

2

* 静态方法：支持编写不预期第一参数为实例对象的方法，支持这类方法向无实例的函数那样运作
* 类方法：调用的时候接收一个类，而不是一个实例，可以用于管理一个类的数据

后续会补充

## 继承

class 语句这样的命名空间的重点就是支持变量名继承 \n 
在 Python 中，对于对象进行点运算符时，就会发生继承，而且涉及了搜索属性定义树（一个或者多个命名空间）。Python 会从头至尾搜索命名空间树，先从对象开始，寻找所能找到的第一个 attr

### 继承方法的专有化
* 系统设计：将系统设计成类的层次，在新增外部的子类对其进行扩展，而不是在原处修改已经存在得了逻辑
* 子类完全可以取代继承的属性，提供超类可以找到的属性，并且通过已经覆盖的方法回调超类来扩展超类的方法

In [42]:
class Super:
    def method(self):
        print('in Super.method')

class Sub(Super):
    def method(self):
        print('starting Sub.method')
        Super.method(self)
        print('ending Sub.method')

### 类接口技术

In [ ]:
class Super:
    def method(self):
        print("in Super.method")

    def delegate(self):
        self.action()


class Inheritor(Super):
    pass


class Extender(Super):
    def method(self):
        print("stating Extender.method")
        Super.method(self)
        print("ending Extender.method")


class Replacer(Super):
    def method(self):
        print("In Replace.method")


class Provider(Super):
    def action(self):
        print("in Provider.action")


if __name__ == "__main__":
    for klass in (Inheritor, Replacer, Extender):
        print("\n" + klass.__name__ + "...")
        klass().method()
        print("\n Provider...")
        x = Provider()
        x.delegate()

### 抽象超类

类的部分行为是由子类所提供的：
1. x.delete 调用中，首先按照继承向上进行查找，直到在 Super 中找到 delegate 方法，并且将 x 传递给这个方法的 self 参数
2. 在 Super.delegate 方法中，self.action 会对 self 以及在他上层的对象启动新的独立继承搜索，self 表示 Provider，并且在 Provider 子类中找到 action 方法

抽象基类在 Python3 中可以通过 @abstractmethod 装饰器进行创建

* 类：class 语句声明的赋值语句。类是产生对象的工厂
* 对象：由类产生的对象
* 实例：self 指的是就是 self 

## 命名空间

可以通过类读取其属性，但是无法从 def 语句外读取函数或方法内的局部变量。
局部变量只有在 def 内的其余代码才是可见的

#### 命名空间字典


* 模块的命名空间实际上是以字典的形式实现的，并且可以由内置属性 __dict__ 显示这一点；属性点运算符其实就是字典的索引运算，属性继承就是搜索链接的字典而已
* 实例和对象就是 Python 中带有链接的字典而已


In [2]:
class Super:
    def hello(self):
        self.data = 'spam'

In [3]:
class Sub(Super):
    def hola(self):
        self.data2 = 'eggs'

实例通过 __class__属性找到其类，类通过 __bases__ 属性找到其超类

In [5]:
x = Sub()

实例的命名空间字典

In [6]:
x.__dict__

{}

实例所属的类

In [7]:
x.__class__

__main__.Sub

类的父类

In [9]:
Sub.__bases__

(__main__.Super,)

In [10]:
Super.__bases__

(object,)

当类为 self 属性赋值时，会填入实例对象。属性最后位于实例的属性命名空间字典内，而不是类的。实例对象的命名空间保存了数据，会随着实例的不同而不同，self 是进入其命名空间的钩子

In [11]:
y = Sub()

In [12]:
x.hello()

In [13]:
x.__dict__

{'data': 'spam'}

In [14]:
x.hola()

In [15]:
x.__dict__

{'data': 'spam', 'data2': 'eggs'}

In [17]:
list(Sub.__dict__.keys())

['__module__', 'hola', '__doc__', '__annotations__']

In [18]:
Super.__dict__.keys()

dict_keys(['__module__', 'hello', '__dict__', '__weakref__', '__doc__'])

In [19]:
y.__dict__

{}

In [20]:
dir(Sub)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'hello',
 'hola']

In [21]:
dir(Super)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'hello']

In [22]:
dir(x)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'data',
 'data2',
 'hello',
 'hola']

# 第二十九章 运算符重载

## 构造函数和表达式：__init__ 和 __sub__

In [35]:
class Number:
    def __init__(self,start):
        self.data = start
    def __sub__(self,other):
        return Number(self.data - other)


In [36]:
x = Number(5)

In [37]:
y = x - 2

In [38]:
y.data

3

## 索引和分片

In [42]:
class Indexer:
    def __getitem__(self, index):
        return index ** 2

In [43]:
x = Indexer()

In [44]:
x[2]

4

In [45]:
for i in range(5):
    print(x[i], end=' ')

0 1 4 9 16 

支持分片

In [46]:
class Indexer:
    data = [5,6,7,8,9]
    def __getitem__(self, index):
        print('getitem', index)
        return self.data[index]
    def __setitem__(self,index,value):
        self.data[index] = value

In [47]:
x = Indexer()

In [48]:
x[0]

getitem 0


5

In [49]:
x[1]

getitem 1


6

In [51]:
x[1:4]

getitem slice(1, 4, None)


[6, 7, 8]

In [52]:
x[-1]

getitem -1


9

通过 __setitem__ 索引赋值方法类似的拦截索引和分片赋值，为后者接受一个分片对象，以同样的方式传递到另一个索引赋值中

* 索引迭代：__getitem__
* 定义该方法可以实现for 循环每次循环时调用类的 __getitem__ ，并持续搭配有更高的偏移值；任何响应索引运算的内置活用户定义的对象，都会响应迭代


In [1]:
class stepper:
    def __getitem__(self,i):
        return self.data[i]

In [2]:
x = stepper()

In [3]:
x.data = 'spam'

In [4]:
for item in x:
    print(item, end=' ')

s p a m 

对于任何支持 for 循环的类都会自动支持 Python 所有的迭代环境；例如：成员关系测试 in、列表解析、内置函数 map、列表和元组赋值运算符以及类型构造方法

In [5]:
'p' in x

True

In [6]:
[c for c in x]

['s', 'p', 'a', 'm']

In [7]:
list(map(str.upper, x))

['S', 'P', 'A', 'M']

In [8]:
list(x)

['s', 'p', 'a', 'm']

In [9]:
tuple(x)

('s', 'p', 'a', 'm')

In [10]:
' '.join(x)

's p a m'

## 迭代器对象：__iter__ 和 __next__

* Python 中的迭代环境都会先尝试 __iter__ 方法，然后尝试 __getitem__ 方法。只有不支持迭代器协议时，才会尝试使用索引运算
* 优先使用 __iter__ 然后尝试使用__getitem__
* 迭代器是通过内置函数 iter 去尝试寻找 __iter__ 方法来实现的，而这种方法应该返回一个迭代器对象。如果已经提供了，Python 就会重复调用这个迭代器的 next 方法，直到触发 Stopiteration 异常；否则改用 __getiterm__ 机制，通过索引偏移量重复索引，直到触发 IndexError 异常


In [13]:
class Squares:
    def __init__(self,start,stop):
        self.value = start - 1
        self.stop = stop
    
    def __iter__(self):
        return self

    def __next__(self):
        if self.value == self.stop:
            raise StopIteration
        self.value +=1
        return self.value ** 2

In [12]:
for i in Squares(1,5):
    print(i, end=' ')

1 4 9 16 25 

In [20]:
x = Squares(1,5)
i = iter(x)

In [21]:
next(i)

1

In [22]:
next(i)

4

__iter__ 机制是在__getitem__ 中所见到的其他所有迭代环境的实现方式（成员关系测试、类构造函数、序列赋值运算）

In [24]:
x = Squares(1,5)

In [25]:
[n for n in x]

[1, 4, 9, 16, 25]

In [26]:
[n for n in x]

[]

#### 多个迭代器对象
*  单迭代对象：生成器函数和表达式、map 以及 zip 内置函数
*  多迭代对象：range 函数、其他内置类型支持多个活跃的迭代器。如果需要实现多个迭代器，__iter__ 只需要定义新的迭代器对象，而不是返回 self

In [8]:
class SkipIterator:
    def __init__(self,wrapped):
        self.wrapped = wrapped
        self.offset = 0
    def __next__(self):
        if self.offset >= len(self.wrapped):
            raise StopIteration
        else:
            item = self.wrapped[self.offset]
            self.offset += 2
            return item

class SkipObject:
    def __init__(self,wrapped):
        self.wrapped =  wrapped
    def __iter__(self):
        return SkipIterator(self.wrapped)
if __name__ == "__main__":
    alpha = "abcedf"
    skipper = SkipObject(alpha)
    I = iter(skipper)
    

In [9]:
next(I),next(I),next(I)

('a', 'c', 'd')

In [10]:
for x in skipper:
    for y in skipper:
        print(x+y,end=' ')

aa ac ad ca cc cd da dc dd 

#### 成员关系：__contaions__、__iter__ 和 __getitem__

* 在迭代领域，类通常把 in 成员关系运算符实现为一个迭代，使用 __item__ 方法或者 __getitem__ 方法，如果要支持更加特定的成员关系，需要编写 __contains__ 方法；
* _contains_方法优先于：_iter_ 方法，__iter__ 方法优先于__getitem__ 方法
* _contains_ 用于将成员关系定义为一个映射应用键（可以快速查找），用于进行序列的搜索


In [11]:
class Iters:
    def __init__(self,value):
        self.data = value
    
    def __getitem__(self,i):
        print('get[%s]: ' % i, end=' ')
        return self.data[i]

    def __iter__(self):
        print('iter=> ', end=' ')
        self.ix = 0
        return self
    def __next__(self):
        print("next:", end='')
        if self.ix == len(self.data): raise StopIteration
        item = self.data[self.ix]
        self.ix += 1
        return item

    def __contains__(self,x):
        print('contains: ', end=' ')
        return x in self.data    

In [12]:
x = Iters([1,2,3,4,5])

In [13]:
print(3 in x)

contains:  True


In [14]:
for i in x:
    print(i, end=' | ')

iter=>  next:1 | next:2 | next:3 | next:4 | next:5 | next:

In [15]:
print([i**2 for i in x])

iter=>  next:next:next:next:next:next:[1, 4, 9, 16, 25]


In [16]:
print(list(map(bin,x)))

iter=>  next:next:next:next:next:next:['0b1', '0b10', '0b11', '0b100', '0b101']


In [17]:
i = iter(x)
while True:
    try:
        print(next(i),end='@')
    except StopIteration:
        break

iter=>  next:1@next:2@next:3@next:4@next:5@next:

分片

In [19]:
x = Iters('spam')
x[0]

get[0]:  

's'

## 属性调用：__getattr__ 和 __setattr__
* _getattr_ 放放是拦截属性点号运算符。对于未定义的属性名称和实例进行点运算时，就会用属性名称作为字符串调用者那个方法
* 如果 Python 可以通过继承树搜索到这个属性，则不会触发该方法的调用。_getattr_ 可以作为钩子来通过通用的方式来响应属性需求

In [23]:
class empty:
    def __getattr__(self,attrname):
        if attrname == "age":
            return 40
        else:
            raise AttributeError(attrname)

In [24]:
x = empty()

In [25]:
x.age

40

In [26]:
x.name

AttributeError: name

__setattr__ 会拦截所有属性的赋值语句。如果定义了这个方法：self.attr = value 就会变成 self.__setattr__('attr',value)；
在 __setattr__ 执行 self 赋值，都是导致再次调用__setattr__，导致无穷递归循环，需要通过属性字典进行索引计算进行赋值：self.__dict__['name'] = x
而不是self.name = x

In [28]:
class accesscontrol:
    def __setattr__(self,attr,value):
        if attr=='age':
            self.__dict__[attr] = value
        else:
            raise AttributeError(attr + ' not allowed')

In [29]:
x = accesscontrol()

In [30]:
x.age = 40

In [31]:
x.age

40

In [32]:
x.name = 'mel'

AttributeError: name not allowed

## 其他属性管理工具

* __getattribute__ 方法拦截所有的属性获取，不仅仅是那些没有定义的，使用时比习更加小心防止出现循环
* Property：内置函数允许我们把方法和特定雷属性上的获取和设置操作关联起来
* 描述符提供了一个协议，把一个类的__get__和__set__方法与特定类属性的方法关联起来

In [51]:
class PrivateExc(Exception): pass
class Privacy:
    def __setattr__(self,attrname,value):
        if attrname in self.privates:
            raise PrivateExc(attrname,self)
        else:
            self.__dict__[attrname] = value

class Test1(Privacy):
    privates = ['age']

class Test2(Privacy):
    privates = ['name','pay']
    def __init__(self):
        self.__dict__['name'] = 'Tom'

In [52]:
x = Test1()
y = Test2()

In [53]:
x.name = 'Bob'

In [54]:
y.name = 'Sue'

PrivateExc: ('name', <__main__.Test2 object at 0x11e103910>)

In [55]:
y.age = 30

In [56]:
x.age = 40

PrivateExc: ('age', <__main__.Test1 object at 0x11e102da0>)

通过上面的方式可以再 Python 中实现属性私有性的首选方法。更建议的方法是通过类装饰器来实现私有属性

#### __repr__ 和 __str__ 会返回字符串表达形式

当实例打印后者转换成字符串时，_repr_ 或者其近亲_str_ 就会自动调用

In [1]:
class adder:
    def __init__(self,value=0):
        self.data = value
    def __add__(self,other):
        self.data += other

In [2]:
x = adder()

In [3]:
print(x)

In [4]:
x

In [5]:
class addrer(adder):
    def __repr__(self):
        return 'addrrepr(%s) ' % self.data

In [6]:
x = addrer(2)
x + 1

In [7]:
x

addrrepr(3) 

In [8]:
print(x)

addrrepr(3) 


In [10]:
str(x),repr(x)

('addrrepr(3) ', 'addrrepr(3) ')

* 打印操作通常尝试_str_和 str 内置函数（print 运行的内部等价形式）。返回用户一个友好的提示
* __repr__ 用于所有其他的环境中：在交互式模式下，用于提示回应，以及 repr 函数，可以用于创建提示，或者给开发者一个更加详细的提示
* 在交互式模式中，只能够实用 __repr__

In [11]:
class addrstr(adder):
    def __str__(self):
        return '[value:%s ]' % self.data

In [12]:
x = addrstr(3)

In [13]:
x + 1

In [14]:
x

In [15]:
print(x)

[value:4 ]


In [16]:
str(x),repr(x)

('[value:4 ]', '<__main__.addrstr object at 0x108d87cd0>')

In [17]:
class addboth(adder):
    def __str__(self):
        return '[Value: %s]' % self.data
    def __repr__(self):
        return '[addboth(%s)]' % self.data

In [18]:
x = addboth(4)

In [19]:
x+1

In [20]:
x

[addboth(5)]

In [21]:
print(x)

[Value: 5]


* str 和 repr 都必须返回字符串，其他结果类型不会被转换，并且会引发错误
* __str__ 有好的用户提示只有当对象出现在一个打印操作顶层的时候才应用。嵌入到大对象中的对象可能用其__repr__ 或者默认的方法打印


In [1]:
class Printer:
    def __init__(self,val):
        self.val = val
    def __str__(self):
        return str(self.val)

In [3]:
objs = [Printer(2),Printer(3)]
for x in objs:
    print(x)

2
3


In [4]:
print(objs)

[<__main__.Printer object at 0x1116d2770>, <__main__.Printer object at 0x1116d2110>]


编写 repr 来适应第二种情况

In [5]:
class Printer:
    def __init__(self,val):
        self.val = val
    def __repr__(self):
        return str(self.val)

In [6]:
objs = [Printer(2),Printer(3)]
for x in objs:print(x)

2
3


In [7]:
print(objs)

[2, 3]


在打印并且可以看到定制显示的时候，肯定是使用了__str__或者__repr__中的任何一个

#### 右侧加法 radd 和 iadd

* 如果需要实现右侧使用实例对象的加法，需要增加__radd__方法。只有当+右侧的对象是类实例，而左边对象不是类实例的时候，会调用__radd__；
* 主要取决于类实例位于 + 左边还是右边


In [16]:
class Commuter:
    def __init__(self, val):
        self.val = val
    def __add__(self,other):
        print('add', self.val, other)
        return self.val + other
        
    def __radd__(self, other):
        print('radd', self.val, other)
        return other + self.val

In [17]:
x = Commuter(88)
y = Commuter(99)

In [18]:
x + 1

add 88 1


89

In [19]:
1 + y

radd 99 1


100

In [20]:
x + y

add 88 <__main__.Commuter object at 0x110c17f10>
radd 99 88


187

类型测试可能需要辨别它是否能够安全的转换并由此避免嵌套 ？？？？

#### 原处加法
为了实现 += 原处相加，编写 iadd 或者 add，如果 iadd 不存在，则使用 add

In [21]:
class Number:
    def __init__(self,val):
        self.val = val
    def __iadd__(self, other):
        self.val += other
        return self

In [22]:
x = Number(5)

In [23]:
x += 1

In [24]:
x+=1

In [25]:
x.val

7

使用 add 实现原处相加

In [30]:
class Number:
    def __init__(self,val):
        self.val =  val
    def __add__(self,other):
        return Number(self.val + other)

In [31]:
x = Number(5)

In [32]:
x += 1

In [33]:
x+=1

In [34]:
x.val

7

## Call 表达式：call

如果定义了 call 表达式，Python 就会为实例应用表达式运行__call__ 方法，让类实例的外观和用法类似于函数

In [42]:
class Callee:
    def __call__(self, *pargs,**kargs):
        print('Called:', pargs, kargs)

In [43]:
c = Callee()

In [44]:
c(1,2,3)

Called: (1, 2, 3) {}


* 带有一个__call__ 的类和实例，支持与常规函数和方法完全相同的参数语法和语义
* 拦截调用表达式允许类实例模拟类似函数的外观，但是也在掉用户总保持了状态信息以供使用


In [54]:
class Prod:
    def __init__(self,value):
        self.value = value
    def __call__(self,other):
        return self.value * other
    def comp(self,other):
        return self.value * other

In [55]:
x = Prod(2)

In [56]:
x(3)

6

In [57]:
x(4)

8

通过一个简单的方法也可以通过上面类似的功能

In [58]:
x.comp(3)

6

In [59]:
x.comp(4)

8

除了 init str repr, call 是最常用的重载方法了

#### 布尔测试： bool 和 len

Python 首先尝试 __bool__ 来获取一个直接的 bool 值，如果没有该方法，尝试根据 __len__ 根据对象的长度确定一个真值

In [60]:
class Truth:
    def __bool__(self):
        return True

In [61]:
x = Truth()

In [63]:
if x:print('yes!')

yes!


通过 len 判断长度是一个费控对象为真

In [64]:
class Truth:
    def __len__(self):
        return 0

In [65]:
x = Truth()

In [67]:
if not x:print('bool')

bool


如果都没有定义，则一定为真

In [68]:
class Truth:
    pass

In [69]:
x = Truth()

In [70]:
bool(x)

True

## 对象析构函数 del
产生实例时，调用 __init__ 构造函数，没当实例被回收时，执行析构函数 __del__ 

In [80]:
class Life:
    def __init__(self,name='unknown'):
        print("hello", name)
        self.name = name
    
    def __del__(self):
        print("GoodBye", self.name)

In [81]:
brain = Life('Brain')

hello Brain


1. 当 brain 赋值为字符串时，我们会失去 Life 实例的最后一个引用，因此出发析构函数，此时需要一些清理行为
2. Python 在实例回收时，会自动回收该实例所拥有的全部空间，对于空间管理不需要析构函数


In [82]:
brain = 'Loretta'

GoodBye Brain


# 第三十章：类的设计

## Python 和 OOP
1. Python 中的 OOP 包含以下三个概念：
2. 继承：继承是基于 Python 中的属性查找实现的
3. 多态：在 x.method 中， method 的意义取决于 x 的类型。实现相同接口的对象是可以相互交换的，客户端不需要知道他们调用的方法的对象种类。多态是基于对象接口的，而不是基于类型的
4. 封装：方法和运算符实现行为，数据隐藏默认是一种惯例；将实现的细节隐藏在对象接口之后。用于保证对象的接口出现变动时，不影响该对象的用户

### 通过标记进行重载

* 一些情况下，OOP 语言 将多态定义为基于参数类型标记的重载函数。但是 Python 中的多态是基于对象接口的，而不是类型
* 应该将代码编写为预期的对象接口，而不是特定的数据类型


## OOP 和 继承 "是一个" 关系

* 程序员的角度：继承是由属性点好运算符启动的，由此触发实例、类、以及任何超累中的变量名搜索
* 设计师的角度：继承是一种定义集合成员关系的方式：类定义了一组内容属性，可由更具体的子类继承和定制

## OOP 和组合："有一个" 关系

* 从程序员的角度：组合涉及把其他对象嵌入容器对象内，并使其实现容器方法。
* 从设计师的角度：组合是另一种表示问题领域中关系的方式

组合反映了各组成部分之间的关系，通常是 has-a 关系。组合类一般都提供自己的接口，并通过内嵌的对象来实现接口

## OOP 和委托： "包装" 对象

* 委托：控制器对象内嵌其他对象，而把运算请求传递给对应的对象。控制器负责管理工作。
* 在 Python 中，委托通常以 __getattr__ 钩子方法来实现，庸医拦截对不存在属性的读取。
* 包装类，代理类：可以使用__getattr__把任意读取转发给被包装的对象


In [1]:
class wrapper:
    def __init__(self, object):
        self.wrapped = object

    def __getattr__(self,attrname):
        print('Trace:', attrname)
        return getattr(self.wrapped, attrname)

In [2]:
x = wrapper([1,2,3])

In [3]:
x.append(4)

Trace: append


In [5]:
x.wrapped

[1, 2, 3, 4]

__getattr__ 会获得属性名称字符串。 改代码利用 getattr 内置函数，以变量名字符串从包裹对象中取出属性：getattr(x,n)就像 x.n，但是 n 是表达式，可以在运算时计算出字符串

In [6]:
x = wrapper({'a':1,'b':2})

In [8]:
list(x.keys())

Trace: keys


['a', 'b']

## 类的伪私有属性

* Python 通过变量名压缩技术实现某些变量局部化。压缩后的变量名有时会被认为是私有属性，但是这其实只是一种把类所创建的变量名局部化的方式而已
* 该功能主要适用于避免实例内的命名空间冲突，而不是限制变量名的读取
* 通过单下换线来编写内部名称: _X

class 语句内开头有两个下换线，但是结尾没有两个下划线的变量名，就会自动扩张，从而包含所在类的名称：
* _x 扩张为：_spam_X：
* 变量名压缩只发生在 class 语句内，而且只针对开头有两个下划线的变量名，对于方法名称和实例属性名称都会发生

在 Python 的类方法内，每当方法赋值 self 的属性时，就会在该实例内修改或创驾该属性。即使在这个层次中有多个类赋值相同的属性

In [16]:
class C1:
    def meth1(self): self.__x = 88
    def meth2(self): print(self.__x)

class C2:
    def metha(self): self.__x = 99
    def methb(self): print(self.__x)

In [17]:
class C3(C1,C2):
    pass

I = C3()


In [19]:
I.meth1();I.metha()
print(I.__dict__)
I.meth2();I.methb()

{'_C1__x': 88, '_C2__x': 99}
88
99


## 方法是对象： 绑定或者无绑定

方法是一种对象，并且用于其他对象大部分相同的方式来广泛使用，支持赋值、将其传递给函数、存储在数据结构中。类方法可以从一个实例或者一个类访问，在 Python 存储字啊以下两种形式：
* 无绑定方法对象（无 self）：
  通过对类进行点号运算符从而获取类的函数属性，会传回无绑定方法对象。调用该方法时，必须明确提供实例对象作为第一个参数。无绑定方法和函数类似，通过类名直接进行调用
* 绑定实例方法对象（self + 函数对）：
  通过对实例进行全运算从而获取类的函数属性，会传回绑定（Bound）方法对象
  Python 在绑定方法对象中自动把实例和函数打包，所以不需要传递实例去调用该方法

In [20]:
class Spam:
    def doit(self,message):
        print(message)

In [21]:
obj1 = Spam()
obj1.doit("hello world")

hello world


绑定方法对象是在过程中产生的，就在方法调用的括号前；
* 可以通过 obj.name 结合运算符称为一个对象表达式，并且将绑定方法赋值给一个变量名后，然后像简单函数那样进行调用

In [22]:
obj1 = Spam()
x = obj1.doit
x("hello world")

hello world


操作无绑定方法对象: 如果对于类进行点号运算符获取属性值，就会得到无绑定方法对象，也就是函数对象的引用值

In [24]:
obj1 = Spam()
t = Spam.doit
t(obj1, 'howdy')

howdy


* 只有通过实例调用，Python 3.0 才会向方法传递一个实例
* 当通过一个类调用的时候，只有在方法期待一个实例的时候，才必须手动传递一个实例

In [25]:
class Selfless:
    def __init__(self,data):
        self.data = data
    
    def selfless(arg1,arg2):
        return arg1 + arg2

    def normal(self, arg1,arg2):
        return self.data + arg1 + arg2

In [26]:
x = Selfless(2)

实例自动传递

In [27]:
x.normal(3,4)

9

In [ ]:
通过方法手动传递实例

In [28]:
Selfless.normal(x,3,4)

9

不传递实例，在 3.0 中也是可以正常工作的

In [30]:
Selfless.selfless(3,4)

7

绑定方法可以像一个通用对象处理，支持像任何其他可调用对象一样对待

In [31]:
class Number:
    def __init__(self, base):
        self.base = base
    def double(self):
        return self.base * 2
    def triple(self):
        return self.base * 3

In [32]:
x = Number(2)

In [36]:
y = Number(3)

In [37]:
z = Number(4)

In [38]:
x.double()

8

In [39]:
acts = [x.double, y.double, y.triple, z.double]
for act in acts:
    print(act())

8
6
9
8


绑定方法对象拥有自己的内省信息，包括让它们配对的实例对象和方法函数访问的属性。调用绑定方法会直接分配配对：

In [40]:
bound = x.double

In [41]:
bound.__self__,bound.__func__

(<__main__.Number at 0x107c5e170>, <function __main__.Number.double(self)>)

In [42]:
bound.__self__.base

4

In [43]:
bound()

8

## 多重继承：混合类

* class 语句中，首行括号中可以列出一个以上的超类，就实现了多重继承：类和实例继承了列出的所有超类的变量名
* 属性搜索时，Python 会由左至右搜索首行中的超类，直到找到相符者
* 传统类：属性的搜索处理对于所有路径都是深度优先
* 新式类：属性沿着树层级进行搜索、以更加广度优先的方式进行搜索


### 编写混合类

In [44]:
class Spam:
    def __init__(self):
        self.data1 = "food"

In [45]:
x = Spam()

In [46]:
print(x)

在混合类中顶一个显示方法一次，使得我们能够在想要看到的一个定制显示格式的任何地方重用

#### 使用 __dict__ 列出实例属性

In [56]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-


class ListInstance:

    def __str__(self) -> str:
        return "<Instance of %s, address %s:\n%s>" % (
            self.__class__.__name__,
            id(self),
            self.__attrnames(),
        )

    def __attrnames(self):
        return "".join(
            "\tname %s=%s\n" % (attr, self.__dict__[attr])
            for attr in sorted(self.__dict__)
        )


* 通过实例的__class__属性获取创建其的类，并通过__name__获取其名称
* 通过扫描实例的属性字典，用于构建一个字符串来显示所有实例属性的名称和值
* 通过调用 ID 显示其内存地址，该函数返回任何对象的地址（根据定义，这是一个唯一的对象标识符号，在随后对这一代码的修改有用）

In [57]:
class Spam(ListInstance):
    def __init__(self):
        self.data1 = 'food'

In [58]:
x = Spam()

In [59]:
print(x)

<Instance of Spam, address 4424928512:
	name data1=food
>


In [60]:
str(x)

'<Instance of Spam, address 4424928512:\n\tname data1=food\n>'

#### 使用 dir 列出继承的属性

__dict__ 混合类只显示实例属性，扩展该类以显式从一个实例可以访问的所有属性，使用 dir 函数，而不是扫描实例的 __dict__ 字典，后者只保存了实例属性

In [62]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-


class ListInstance:

    def __str__(self) -> str:
        return "<Instance of %s, address %s:\n%s>" % (
            self.__class__.__name__,
            id(self),
            self.__attrnames(),
        )

    def __attrnames(self):
        result = ''
        for attr in dir(self):
            if attr[:2] == '__' and attr[-2:] == '__':
                result += '\tname %s =<>\n ' % attr
            else:
                result += '\tname %s=%s\n' %(attr, getattr(self, attr))

        return result


#### 列出类树种每个对象的属性

## 类是对象：通用对象的工厂

* 基于类的设计要求创建的对象来响应条件，而这些条件是在编写程序的时候无法预料的。工厂设计模式允许这样一种延迟方法
* 类是对象，因此很容易在程序中进行传递，保存在数据结构中，也可以把类传递给会产生任意种类对象的函数。这类函数在 OOP 设计领域中偶尔称为工厂


In [2]:
def factory(aClass,*args,**kwargs):
    return aClass(*args,**kwargs)

class Spam:
    def doit(self,message):
        print(message)

class Person:
    def __init__(self,name,job):
        self.name = name
        self.job = job

In [3]:
obj1 = factory(Spam)
obj2 = factory(Person,"Guido",'guru')

## 为什么要有工厂

工厂函数可以将代码和动态配置的对象的构造细节隔离开

## 与设计相关的其他话题
* 继承
* 复合
* 委托
* 多继承
* 绑定方法
* 工厂
* 抽象基类
* 装饰器
* 类型子类
* 静态方法和类方法
* 管理属性
* 元类

# 第三十一章：类的高级主题

## 扩展内置类型
类可以用于扩展 Python 内置类型的功能，从而支持更另类的数据结构

### 通过嵌入扩展类型
通过将列表嵌入到新类中，并且新增运算符重载，实现了新的集合对象

In [2]:
class Set:
    def __init__(self,value=[]):
        self.data = []
        self.concat(value)

    def interset(self,other):
        res = []
        for x in self.data:
            if x in other:
                res.append(x)
        return Set(res)

    def union(self,other):
        res = self.data[:]
        for x in other:
            if not x in res:
                res.append(x)
        return Set(res)

    def concat(self,value):
        for x in value:
            if not x in self.data:
                self.data.append(x)

    def __len__(self): return len(self.data)
    def __getitem__(self,key): return self.data[key]
    def __and__(self,other): return self.interset(other)
    def __or__(self,other): return self.union(other)
    def __repr__(self): return "Set:" + repr(self.data)


调用新类的方法

In [3]:
x = Set([1,3,5,7])

In [4]:
print(x.union(Set([1,4,7])))

Set:[1, 3, 5, 7, 4]


In [5]:
print(x | Set([1,4,6]))

Set:[1, 3, 5, 7, 4, 6]


### 通过子类扩展类型
* 所有的内置类型都能够直接创建子类，这种行为支持用户通过自定义的 class 语句定制或者扩展内置类型：类型的子类能够出现在原始内置类型出现的任何地方
* 定制自己的子类来修改定制列表的核心行为


In [13]:
class MyList(list):
    def __getitem__(self,offset):
        print('(indexing %s at %s)' % (self,offset))
        return list.__getitem__(self, offset-1)    

In [14]:
print(list('abc'))

['a', 'b', 'c']


In [15]:
x = MyList('abc')
print(x)

['a', 'b', 'c']


In [16]:
print(x[1])

(indexing ['a', 'b', 'c'] at 1)
a


In [17]:
print(x[3])

(indexing ['a', 'b', 'c'] at 3)
c


In [18]:
x.append('spam')

In [19]:
x.reverse()

In [20]:
x

['spam', 'c', 'b', 'a']

通过继承子类的方式来定制内置类型，是很强大的工具

In [25]:
class Set(list):
    def __init__(self, value=[]):
        list.__init__([])
        self.concat(value)

    def intersect(self,other):
        res = []
        for x in self:
            if x in other:
                res.append(x)

        return Set(res)

    def union(self,other):
        res = Set(self)
        res.concat(other)
        
        return res
    
    def concat(self,value):
        for x in value:
            if x not in self:
                self.append(x)
    def __and__(self,other): return self.intersect(other)
    def __or__(self, other): return self.union(other)
    def __repr__(self): return 'Set:' + list.__repr__(self)

In [26]:
x = Set([1,3,5,7])
y = Set([2,1,4,5,6])

In [31]:
print(x,y,len(x),end='\t')

Set:[1, 3, 5, 7] Set:[2, 1, 4, 5, 6] 4	

In [32]:
print(x.intersect(y),y.union(x))

Set:[1, 5] Set:[2, 1, 4, 5, 6, 3, 7]


In [33]:
print(x & y, x | y)

Set:[1, 5] Set:[1, 3, 5, 7, 2, 4, 6]


In [34]:
x.reverse();print(x)

Set:[7, 5, 3, 1]


## 新式类
在 Python 2 中存在新式类和经典类的区分，但是在 3.0 中已经基本上融合了
* Python 3.0 中，无论是否显式的继承自 Object。所有的实例的都默认继承 Object，并且所有的对象都是 Object 的实例
* 在 Python 2.6 和以前的版本中，类必须继承自的类看做是新式 Object，并且获得所有新式类的特征

新式类在语法和行为上几乎与经典类完全向后兼容，只是添加了一些高级的新特性而已

### 新式类的变化

继承搜索顺序
* 多继承的钻石模式有一种略微不同的搜索顺序。先横向搜索再纵向搜索，并且先宽度优先搜索，再深度优先搜索

新式类支持一些新的高级工具
* 新式类：slow、特征、描述符和__getattribute 方法

针对内置函数的属性获取
__getattr__ 和 __getattribute__ 方法不再针对内置运算的隐式属性获取而运行。名称搜索从类开始，而不是从实例开始

## 类模式变化
类自身就是类型：
1. Python3 中的所有类都是自动新式类，即使他们没有显式的超类。类本身就是一个类类型
2. 经典类中的实例类型都具有相同的实例类型，要比较类型，必须要比较__class__ 属性。 新式类中，比较实例类型会自动比较实例的类


所有的类隐式的派生自（继承自）Object 内置类，不管是直接的还是间接的通过一个超类

In [35]:
class C:pass

In [36]:
x = C()

In [37]:
type(x)

__main__.C

In [38]:
type(C)

type

每个类都有一个显式或者隐式的超类

In [39]:
isinstance(x,object)

True

In [41]:
isinstance(C,object)

True

#### 钻石继承变动

* 经典类：继承搜索程序是绝对深度优先，然后是从做至右。Python 一路向上搜索，深入树的左侧，返回后才开始深入树的右侧
* 新式类：搜索为宽度优先。Python 先查找第一个搜索的右侧的所有超类，然后一路向上搜索至顶端共同的超类，先水平搜索再向上移动。新式规则可以再多个子类访问超类的时候，可以避免访问同一个超类

##### 钻石继承的例子

In [42]:
class A:
    attr = 1
class B(A):
    pass
class C(A):
    pass

In [43]:
class D(B,C):
    pass

In [44]:
x = D()

In [45]:
x.attr

1

* 经典类查找顺序：搜索顺序为D、B、A、C（在 A 中找到时，B 之上的搜索就会停止）
* 新式类的查找顺序：D、C、B、A

In [49]:
class A(object):
    attr = 1

class B(A):
    pass

class C(A):
    attr = 2
    pass

class D(B,C):
    pass

In [50]:
x = D()

In [51]:
x.attr

2

明确继承的属性来解决可能遇到的冲突问题
通过赋值或者在类的混合处，指出你想要的变量名

In [53]:
class A:
    attr = 1
class B(A):
    pass

class C(A):
    attr = 2

class D(B,C):
    attr = C.attr

In [54]:
x = D()

In [55]:
x.attr

2

通过上述的方式来解决冲突，大致上就能够忽略搜索顺序的差异，而不依赖假设来决定所编写的类的意义

方法是可以正常赋值的对象，也可以使用上述的方式解决

In [63]:
class A:
    def meth(s): print('A.meth')

class C(A):
    def meth(s): print('C.meth')

class B(A):
    pass

class D(B,C):
    meth = C.meth
    pass

In [64]:
x = D()

In [65]:
x.meth()

C.meth


#### 搜索顺序变化的范围
新式类中的隐式 Object 超类为各种内置操作提供了默认方法，包括 __str__和 __repr__ 显示格式化方法

In [66]:
dir(object)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

## 新式类的扩展

### slots 实例
* 将字符串属性名称顺序赋值给特殊的__slots__类属性，新式类就有可能既限制类的实例的合法属性集，又能够优化内存和速度的性能


该属性一般是在 class 语句顶层内将字符串名称顺序赋值给变量 __slots__ 而设置；只有__slots__列表内的这些变量名可赋值为实例属性，实例属性名必须在引用前赋值

In [67]:
class limiter(object):
    __slots__ = ['age','name','job']

In [68]:
x = limiter()

In [69]:
x.age = 40

In [70]:
x.age

40

In [71]:
x.ape = 1000

AttributeError: 'limiter' object has no attribute 'ape'

* slot 对于 Python 动态特性来说是一种违背，而动态特性要求任何名称都可以通过赋值来创建
* slot 属性可以顺序存储以供快速查找，而不是为每个实例分配一个字典
* 只有 slot 列表中的名称可以分配给实例，但是，基于 slot 的属性仍然可以使用通用工具通过名称来访问或者设置

In [72]:
class C:
    __slots__ = ['a','b']

In [73]:
x = C()

In [74]:
x.a=1
x.a

1

In [75]:
x.__dict__

AttributeError: 'C' object has no attribute '__dict__'

In [76]:
getattr(x, 'a')

1

In [77]:
setattr(x,'b',2)

In [78]:
'a' in dir(x)

True

In [79]:
'a' in dir(x)

True

In [80]:
'b' in dir(x)

True

如果没有一个属性命名空间字典，不可能给不是 slots 列表中名称的实例来分配新的名称

In [89]:
class D:
    __slots__ = ['a','b','__dict__']
    c = 3
    def __init__(self): self.d = 4

In [90]:
x = D()

In [91]:
x.d

4

In [92]:
x.__slots__

['a', 'b', '__dict__']

In [ ]:
通过在 slot 中增加 __dict__ 是可以容纳额外的属性的

In [93]:
x.c

3

In [98]:
x.a = 1

In [99]:
getattr(x,'a'),getattr(x,'c'),getattr(x,'d')

(1, 3, 4)

In [100]:
x.b = 1

如果需要查找处对应的属性，需要覆盖这两种存储方式，以为 dir 也返回继承的属性

In [101]:
for attr in list(x.__dict__) + x.__slots__:
    print(attr,'==>', getattr(x,attr))

d ==> 4
a ==> 1
b ==> 1
__dict__ ==> {'d': 4}


#### 超类中的多个 __slots__ 列表

## 类特性

* property 提供另一种方式让新式类定义自动调用的方法，来读取或赋值实例属性。
* property 和 __getattr__ 和 __setattr__ 重载方法的替代做法，只在读取所需要的动态计算的变量名时，才会发生额外的方法调用

property 是一种对象，赋值给类属性名称。property 的产生是以三种方法（获得、设置以及删除运算的处理器）以及通过文档字符串调用内置函数 preperty

In [102]:
class newprops(object):
    def getage(self):
        return 40

    age = property(getage,None,None,None) # GET SET DEL DOCS

In [103]:
x = newprops()

In [104]:
x.age

40

property 比传统技术不是那么复杂，而且运行起来更快。当我们新增属性赋值运算支持时，property 就会更加有吸引力

In [1]:
class newsrepos(object):
    def getage(self):
        return 40

    def setage(self,value):
        print('set age', value)
        self._age = value
    
    age = property(getage,setage,None,None)

In [2]:
x = newsrepos()

In [3]:
x.age 

40

In [4]:
x.age = 42

set age 42


In [6]:
x.name = 'trainer'

In [7]:
x.name

'trainer'

#### __getattribute__ 和描述符
* __getattribute__ 方法只适合于新式类，可以让类拦截所有属性的引用，而不局限于未定义的引用
* Python 支持描述符的概念 -- 带有 __get__ 和 __set__ 方法的类，分配给类属性，并且由实例继承，用于拦截对于特定属性的读取和访问


## 元类


## 静态方法和类方法


Python 2.2 中可能存在两种方法：
1. 静态方法大致与一个类中的简单的无实例函数类似的工作
2. 类方法传递一个类而不是一个实例
3. 静态方法和类方法只对经典类有效

1. Python 2.2 中使用上述这些算法必须在类中调用特殊的内置函数：staticmethod和 classmethod、或者使用装饰器语法
2. Python3 中，无实例的方法只通过一个类名来调用，不需要 staticmethod 声明

## 为什么使用特殊方法
1. 程序需要处理与类而不是与实例相关的数据。需要类中的一个方法不仅不传递而且页不期待一个 self 实例参数。
2. Python 可以通过静态方法来支持这样的目标，并且旨在操作类属性而不是实例属性。静态方法不会接受一个自动的 self 参数，不管是通过一个类还是一个实例调用。通常用于记录所有实例的信息，而不是为实例提供行为
3. Python 还支持类方法的概念，这个是类的一种方法，传递给他们的第一个参数是一个类对象而不是一个实例

### Python2.0 和 Python3 中的静态方法
1. 在 Python 2.6 中，从一个类获取一个方法会产生一个未绑定方法，没有手动传递一个实例就不会调用它。无论通过实例或者类调用，总要求传入一个实例
2. 在 Python 3.0 中，从一个类获取一个方法会产生一个简单函数，没有给出实例也可以常规调用。只有方法期待实例调用的时候，才传入实例

通过类属性计算从一个类产生了多少实例：
1. 把类的计数器存储为类属性，每次创建一个新的实例时，构造器函数 + 1
2. 支持显示计数器

In [1]:
class Spam:
    numInstances = 0
    def __init__(self):
        Spam.numInstances = Spam.numInstances + 1
    def printNumInstances():
        print("Number of instances created: ", Spam.numInstances)

函数：printNumInstances 用于处理所有的类数据而不是实力数据，是关于所有实例的，而不是某个特定的实例

In [6]:
a=Spam();b=Spam();c=Spam()

In [7]:
Spam.printNumInstances()

Number of instances created:  3


上述代码在 Python3 中是可以执行的，但是在 Python2 中是不能够执行的

In [8]:
a.printNumInstances()

TypeError: Spam.printNumInstances() takes 0 positional arguments but 1 was given

### 静态方法替代方案
如果希望调用没有一个实例而访问类成员的函数，最简单的思路就是仅在类之外生成它们的简单函数，而不是类方法; 以下函数在 Python3.0 和 Python2.6 中都能够遇到

In [10]:
def printNumInstances():
    print("Number of instances created", Spam.numInstances)

class Spam:
    numInstances = 0
    def __init__(self):
        Spam.numInstances += 1 

In [11]:
a = Spam();b = Spam(); c = Spam()

In [12]:
printNumInstances()

Number of instances created 3


1. 通过将函数名变为全局变量，使得上述方法可以通用工作；
2. 给文件的作用域增加了一个额外的名称，该名称只用来处理单个的类
3. 该函数与类的关联性很小，这样的方法没法通过继承修改

### 使用静态和类方法

1. 编写和类相关联的简单函数
2. 使用静态和类方法编写类，需要调用 staticmethod 和 classmethod

1. 对于 staticmethod 不需要传递实例
2. 对于 classmethod 需要一个类参数

In [18]:
class Method:
    def imeth(self, x):
        print(self, x)

    def smeth(x):
        print(x)

    def cmeth(cls, x):
        print(cls, x)

    semth = staticmethod(smeth)
    cmeth = classmethod(cmeth)

Python 支持三种类相关的方法：实例、静态和类。Python 3.0 也允许类中的简单函数在通过一个类调用的时候充当静态方法的角色，而不需要额外的协议，从而扩展了这一模式
* 实例方法：必须通过实例对象调用实例方法。通过实例调用时，Python 会把实例自动传给第一个参数。类调用时，需要手动传递传入实例
* 静态方法：静态方法调用时不需要实例参数。与类之外的简单函数不同，其变量名位于定义所在累的范围内，属于局部变量，可以通过继承查找
* 类方法：Python 自动将类而不是实例传入类方法的第一个参数中，不管是通过一个类还是实例调用

In [22]:
obj = Method()

In [23]:
obj.imeth(1)

<__main__.Method object at 0x107d87d30> 1


In [24]:
Method.imeth(obj,2)

<__main__.Method object at 0x107d87d30> 2


In [25]:
Method.cmeth(5)

<class '__main__.Method'> 5


In [26]:
obj.cmeth(6)

<class '__main__.Method'> 6


## 使用静态方法统计实例
用于将方法标记为特殊的，以便于不会自动传递一个实例

In [27]:
class Spam:
    numInstances = 0
    def __init__(self):
        Spam.numInstances += 1
    def printNumInstances():
        print("Number of instances: ", Spam.numInstances)
    printNumInstances = staticmethod(printNumInstances)

使用静态方法内置函数可以通过类或者其任何实例来调用无 self 方法

In [28]:
a = Spam(); b = Spam(); c = Spam()

In [30]:
Spam.printNumInstances()

Number of instances:  3


In [31]:
a.printNumInstances()

Number of instances:  3


将函数名称作为类作用域内的局部变量，而且吧函数程序代码靠近其使用的地方，并且支持子类用继承的方式定制静态方法

In [32]:
class Sub(Spam):
    def printNumInstances():
        print("Extra stuff ....")
        Spam.printNumInstances()
    printNumInstances = staticmethod(printNumInstances)

In [33]:
a = Sub()

In [34]:
b = Sub()

In [35]:
a.printNumInstances()

Extra stuff ....
Number of instances:  5


In [36]:
Sub.printNumInstances()

Extra stuff ....
Number of instances:  5


In [37]:
Spam.printNumInstances()

Number of instances:  5


### 实用类方法统计实例
* 类方法：使用一个类方法把实例的类接收到其第一个参数中。类方法使用通用的自动传递类对象

In [45]:
class Spam:
    numInstances = 0 
    def __init__(self):
        Spam.numInstances +=1
    def printNumInstances(cls):
        print("Number of instances: ", cls.numInstances)
    printNumInstances = classmethod(printNumInstances)

可以类和实例调用 PrintNumInstance 的时候，接收的参数是类而不是实例

In [46]:
a,b = Spam(),Spam()

In [47]:
a.printNumInstances()

Number of instances:  2


In [48]:
Spam.printNumInstances()

Number of instances:  2


In [50]:
class Spam:
    numInstances = 0
    def __init__(self):
        Spam.numInstances += 1
    def printNumInstances(cls):
        print("Number of Instances", cls.numInstances, cls)
    printNumInstances = classmethod(printNumInstances)

class Sub(Spam):
    def printNumInstances(cls):
        print("Extra stuff ...", cls)
        Spam.printNumInstances()
    printNumInstances = classmethod(printNumInstances)

class Other(Spam): pass

无论何时运行一个类方法的时候，最低层的类传入，即使自己的类没有对应的类方法

In [51]:
x,y = Sub(),Spam()

In [52]:
x.printNumInstances()

Extra stuff ... <class '__main__.Sub'>
Number of Instances 2 <class '__main__.Spam'>


In [53]:
Sub.printNumInstances()

Extra stuff ... <class '__main__.Sub'>
Number of Instances 2 <class '__main__.Spam'>


In [54]:
y.printNumInstances()

Number of Instances 2 <class '__main__.Spam'>


Other 子类没有任何方法， cls 依然将其进行传入

In [55]:
z = Other()

In [56]:
z.printNumInstances()

Number of Instances 3 <class '__main__.Other'>


#### 使用类方法统计每个类的实例
* 类方法总是接收一个实例树中的最低类：
* * 静态方法和显示类名称对于处理一个类本地的数据来说是最好的解决方案
  * 类方法更适合处理对层级中每个类不同的数据

In [65]:
class Spam:
    numInstances = 0 
    def count(cls):
        cls.numInstances += 1
    def __init__(self):
        self.count()
    count = classmethod(count)

class Sub(Spam):
    numInstances = 0
    def __init__(self):
        Spam.__init__(self)

class Other(Spam):
    numInstances = 0

In [66]:
x = Spam()

In [67]:
y1,y2 = Sub(),Sub()

In [68]:
z1,z2,z3 = Other(),Other(),Other()

In [69]:
x.numInstances,y1.numInstances,z1.numInstances

(1, 2, 3)

In [70]:
Spam.numInstances,Sub.numInstances,Other.numInstances

(1, 2, 3)

## 装饰器和元类：第一部分
函数装饰器提供了一种方式，替函数明确了特定的预算模式，将函数包裹了另一层，在另一函数的逻辑内实现

### 函数装饰器基础
函数装饰器是它后边的函数的运行时的声明。调用方法函数的名称，实际上是触发了它 staticmethod 装饰器的结果，装饰器可以返回原始函数，或者新的对象

通过装饰器对于代码进行修改

In [71]:
class Spam:
    numInstances = 0 
    def __init__(self):
        Spam.numInstances += 1
    @staticmethod
    def printNumInstances():
        print("Number of instances created: ", Spam.numInstances)

In [72]:
a = Spam()
b = Spam()
c = Spam()

通过类和实例调用都是有效的

In [73]:
Spam.printNumInstances()

Number of instances created:  3


In [74]:
a.printNumInstances()

Number of instances created:  3


## 装饰器例子
在实例中存储装饰器的类，并捕捉对最初变量名的调用

In [81]:
class tracer:
    def __init__(self,func):
        self.calls = 0
        self.func = func

    def __call__(self,*args):
        self.calls += 1
        print("call %s to %s" % (self.calls, self.func.__name__))
        self.func(*args)


In [82]:
@tracer
def spam(a,b,c):
    print(a,b,c)

In [83]:
spam(1,2,3)

call 1 to spam
1 2 3


In [84]:
spam('a','b','c')

call 2 to spam
a b c


In [85]:
spam(4,5,6)

call 3 to spam
4 5 6


### 类装饰器和元类
类装饰器类似于函数装饰器，在 class 语句的默认执行，并且把一个类名重新绑定到一个可调用对象，通常用于在类创建之后管理类，或者在创建实例后插入一个包装逻辑层来管理实例

通过钩子自动扩展带有实例计数器和任何其他所需数据的类

In [86]:
def count(aClass):
    aClass.numInstances = 0 
    return aClass

@count
class Spam: pass

@count
class Sub(Spam):pass

@count
class Other(Spam):pass


元类：与类装饰器的功能有所重合。提供了一种可选的模式，会把一个类对象的创建导向顶级 type 类的一个子类

In [87]:
class Meta(type):
    def __new__(meta,classname,supers,classdict):
        pass

class C(metaclass=Meta):pass

后续补充元类的知识

## 类陷阱
类本质上是多了一些技巧的命名空间而已

### 修改类属性的副作用

In [88]:
class X:
    a = 1


In [89]:
i = X()

In [90]:
i.a

1

In [92]:
X.a

1

修改了 x 中的 a，会导致 X 的实例 I 也会被修改

In [95]:
X.a = 2

In [96]:
i.a

2

*为了避免发生上述问题，修改实例，而不是修改类*

### 修改可变的类属性可能产生副作用

类属性由所有实例共享，如果一个类属性引用一个可变对象，那么从任何实例原处修改该对象都会立即影响所有实例

In [97]:
class C:
    shared = []
    def __init__(self):
        self.perobj = []

In [98]:
x = C()

In [99]:
y = C()

In [100]:
x.shared, y.perobj

([], [])

In [101]:
x.shared.append('spam')
x.perobj.append('spam')

In [102]:
x.shared,x.perobj

(['spam'], ['spam'])

In [103]:
y.shared,y.perobj

(['spam'], [])

In [104]:
C.shared

['spam']

### 多重继承：继承顺序很重要

Python 总是会根据超类在首行中的顺序，由左到右搜索超类

可以在子类中指定需要继承的父类的属性，避免多重继承选择的属性不符合自己的预期

#### 过度包装

除非真的有必要，否则不要把事情弄得很复杂。抽象是多态和封装的基础，只要恰当的使用就会成为非常高效的工具

# 第三十二章：异常处理

可以在任意一个步骤内，跳转至异常处理器，中止开始的所有函数调用而进入异常管理器。在一场管理器中编写代码，响应处理异常

异常的角色：
1. 错误处理：在程序代码中捕捉和响应错误，进入异常处理器处理异常
2. 事件通知：可以用于发出有效的信号，而不需啊哟在程序间传递结果标志位
3. 特殊情况处理：
4. 终止行为：
5. 非常规控制流程：

出发异常的语句：
1. try except
2. try finally
3. raise
4. assert
5. with / as

如果没有捕捉异常，就会一直向上返回到程序顶层，并且启用默认的异常处理器，打印出标准的出错消息

try except 支持捕捉异常，也可以从中恢复执行

In [105]:
def cacher():
    try:
        fetcher(x,4)
    except IndexError:
        print('got  exception')
    print('continuing')

### 引发异常

### 用户定义的异常

In [106]:
class Bad(Exception):
    pass

In [107]:
def doomed():
    raise Bad()

In [108]:
try:
    doomed()
except Bad:
    print("got Bad")

got Bad


#### 终止行为

无论是否发生了异常，可以用于定义一定会在最后执行时收尾的行为

In [109]:
try:
    pass
finally:
    print("after fetch")

after fetch


# 第三十三章：异常编码细节

try 语句的运行方式：
1. 如果 try 代码块语句执行时发生了异常，Python 跳回 try，执行第一个符合引发异常的 except 子句下的语句，执行后，控制权会到整个 try 语句后继续执行
2. 如果 try 代码块发生异常，但是没有符合的 except 子句，异常就会向上传递到程序之前进入 try 中，如果现在是第一条，就会传递到顶层
3. 如果 try 首行底下执行的语句没有发生异常，Python 就会执行 else 行下的语句

else 语句只有在不发生异常时才会执行

捕捉所有异常

In [1]:
try:
    action()
except :
    pass
else:
    pass

上述的异常捕捉，会捕捉到很多与程序代码无关的异常，比如系统异常，而且可能拦截其他处理器的异常；Python 3.0 中引入 Exceptiton 用于忽略和系统退出相关的异常

### try/else 语句

else 语句 用于判断控制流程是否通过了 try 语句 

In [2]:
try:
    run code
except IndexError:
    handle exception
else:
    no exception occured

SyntaxError: invalid syntax (275941454.py, line 2)

异常的默认行为：打印

In [3]:
def gobad(x,y):
    return x / y

def gosouth(x):
    print(gobad(x,0))
gosouth(1)

ZeroDivisionError: division by zero

### try finally 语句

如果 try 语句中包含了 finally ，Python 一定会在 try 语句后执行其语句代码块，无路 try 是否发生了异常

In [4]:
try:
    <statement>
finally:
    <statement>

SyntaxError: invalid syntax (2725364301.py, line 2)

1. 抛出异常：执行 try + finally 然后再抛出异常
2. 未抛出异常：执行 finally 然后在整个 try 语句后继续执行下去

利用 try finally 编写终止行为

In [6]:
class MyError(Exception):pass

def stuff(file):
    raise MyError()

file = open('data','w')
try:
    stuff(file)
finally:
    file.close()
    print('not reached')

not reached


MyError: 

## raise 语句

显示的触发异常语句，组成：raise 关键字，后面跟着可选的要引发的类或者一个类的实例

In [ ]:
raise <instance>
raise <class>

## assert 语句

In [7]:
assert <test>,<data>

SyntaxError: invalid syntax (530231791.py, line 1)

如果 test 为假，则抛出异常

## 环境管理协议

with 语句实际的工作方式：
1. 计算表达式，所得到的对象称为环境管理器，必须包含 __enter__ 和 __exit__ 方法
2. 环境管理器的__enter__方法会被调用。如果 as 子句存在，其返回值就会赋值给 As 子句中的变量，否则丢弃
3. 代码块中嵌套的代码会执行
4. 如果 with 代码块引发异常，__exit__ （type、value、traceback）方法就会被调用。如果此方法为假，则异常会重新引发，否则，异常会终止
5. 如果 with 代码块没有引发异常，__exit__ 方法依然会被调用，其 type、value 以及 traceback 参数都会以 None 传递

In [8]:
class TraceBlock:
    def message(self,arg):
        print('running', arg)
    def __enter__(self):
        print("starting with block")
    def __exit__(self,exc_type,exc_value,exc_tb):
        if exc_type is None:
            print("exited normally\n")
        else:
            print("raise an exception!!", exc_type)
            return False

In [10]:
with TraceBlock() as action:
    action.message('test 2')
    print('reached')

starting with block
raise an exception!! <class 'AttributeError'>


AttributeError: 'NoneType' object has no attribute 'message'

# 第三十四章: 类异常

使用类异常并且

In [1]:
class Numerr(Exception):pass
class Divzero(Numerr):pass
class Oflow(Numerr):pass

In [ ]:
try:
    action()
except Numerr:
    pass

## 内置的 Exception 类

#### 定制异常的打印显示

In [2]:
class MyBad(Exception):
    def __str__(self):
        return "Always look on bright side of life"

In [3]:
try:
    raise MyBad()
except MyBad as x:
    print(x)

Always look on bright side of life


## 定制数据和行为

通过自己提供构造函数，类可以定义在处理器中的方法，来提供预先编码的异常处理逻辑

In [4]:
class FormatError(Exception):
    def __init__(self,line,file):
        self.line = line
        self.file = file

def parser():
    raise FormatError(42,file='spam.txt')

In [5]:
try:
    parser()
except FormatError as x:
    print('Error at', x.file,x.line)

Error at spam.txt 42


# 高级话题：Unicode 和 字节字符串

## 字符串基础知识

### 字符编码方法

ASCII 定义了从 0 到 127 的字符代码，并且允许每个字符存储在意一个 8 位的字节中，存储在内存和文件的一个单个字节中

Python 的内置函数 ord 给出一个字符的二进制值

In [1]:
ord('a')

97

In [2]:
hex(97)

'0x61'

针对给定的整数代码值返回其字符串

In [4]:
chr(97)

'a'

Unicode 文本通常叫做 __宽字符__ 字符串，因为每个字符可能表示为多个字节。用于表示 8 为字符更多的字节。在计算机中存储丰富的文本，需要保证字符和原始字节之间可以使用一种编码进行相互转换。编码表示就是把一个 Unicode 字符转换为字节序列，以及从一个字节序列提取字符串的规则
* 编码：根据一个想要的编码名称，把一个字符翻译为其原始的字节形式
* 解码：根据其名称，把一个原始字符串翻译为字符串形式的过程
由于编码的字符映射吧字符分配给同样的代码以保证兼容性，因此 ASCII 是 Latin-1 和 UTF-8 的子集

utf-8：通过采用可变的字节数的方案，允许表示众多的字符
1. 小于 128 的字符代码表示为单个字节
2. 128--0x7ff 之间的代码转换为两个字节，每个字节拥有一个 128 到 255 之间的值
3. 0x7ff 以上的代码转换为 3 个或者 4 个字节序列

In [1]:
0xC4

196

In [2]:
chr(196)

'Ä'

* 在 Python 2.6 中 我们可以对简单的文本使用 str 并对文本的更高级的形式使用二进制数据 和 Unicode
* 在 Python 3.0 中 我们将针对任何类型的文本使用 str，并且针对二进制数据使用 bytes 和 bytearray

In [1]:
import sys
sys.getdefaultencoding()

'utf-8'

In [2]:
import encodings
help(encodings)

Help on package encodings:

NAME
    encodings - Standard "encodings" Package

DESCRIPTION
        Standard Python encoding modules are stored in this package
        directory.
    
        Codec modules must have names corresponding to normalized encoding
        names as defined in the normalize_encoding() function below, e.g.
        'utf-8' must be implemented by the module 'utf_8.py'.
    
        Each codec module must export the following interface:
    
        * getregentry() -> codecs.CodecInfo object
        The getregentry() API must return a CodecInfo object with encoder, decoder,
        incrementalencoder, incrementaldecoder, streamwriter and streamreader
        attributes which adhere to the Python Codec Interface Standard.
    
        In addition, a module may optionally also define the following
        APIs which are then used by the package's codec search function:
    
        * getaliases() -> sequence of encoding name strings to use as aliases
    
        Ali

## Python 的字符串类型

Python3 中支持三种字符串字符串对象
* str 表示 Unicode 文本（8 位的和更宽的）
* bytes 表示二进制数据。定义了一个 8 位整数的不可变序列，表示绝对的字节值。该类型支持几乎 str 类型所做的所有相同操作：字符串方法、序列操作、re 模块匹配
* bytearray 是一种可变的 bytes 类型

__bytes__
Python 3.0 的 bytes 对象其实只是较小整数的一个序列，每个整数的范围都在 0-255 之间；索引一个 bytes 将返回一个 int；分片一个 bytes 将返回另一个 bytes；在一个 bytes 上运行内置函数 list 将返回整数，而不是一个字符串的列表

In [3]:
B = b'spam'

In [4]:
B[0],B[:1],list(B[:2])

(115, b's', [115, 112])

In [5]:
ord('s')

115

bytearray 类型，bytearray 是 bytes 类型的一个变体，是可变的并且支持原处修改。支持 str 和 bytes 所支持的常见的字符串操作，以及和列表相同的很多原处修改。Bytearray 为字符串增加了直接原处修改的能力

Python 3.0 中，针对任何的文本使用 str，并且针对任何的二进制数据使用 bytes 和 bytearray

## 文本文件和二进制文件

* 文本文件：当一个文件以文本模式打开时，读取其数据会自动将其内容解码为，并且将其返回为一个 str，写入接受 str，并且在写入文件前进行编码
* 二进制文件：通过内置 open 打开文件时，添加 b，以二进制模式打开一个文件，读取文件时不会以任何方式进行解码，直接返回 raw 内容，并且作用 bytes 对象。二进制文件接受一个 Bytearray 对象作为写入文件中的内容

### Python 3.0 中的字符串应用

创建 字符串 byte 和 Bytearray

In [8]:
s = 'spam'
b = b'spam'
b_array = bytearray('spam',encoding='utf-8')

In [9]:
type(s),type(b),type(b_array)

(str, bytes, bytearray)

bytes 对象实际上激素是较小的整数序列，可以将自己的内容打印为字符

In [11]:
b[0],s[0]

(115, 's')

byte 进行切片

In [13]:
b[1:3],s[1:3]

(b'pa', 'pa')

调用 list

In [14]:
list(s),list(b)

(['s', 'p', 'a', 'm'], [115, 112, 97, 109])

bytes 对象和 str 一样是不可以修改的，Bytearray 则是可以进行修改的

In [15]:
b[0] = 'x'

TypeError: 'bytes' object does not support item assignment

### 转换

Python3 引入了更加鲜明的区分：str 和 bytes 类型对象不在表达式中自动混合，传递给函数的时候不会自动的进行相互转换。
转换：
1. str.encode() 和 bytes(s,encoding) 用于将一个字符串转换为其 raw bytes 形式，并且在此过程中根据一个 str 创建一个 bytes
2. bytes.decode() 和 str(B,encoding) 把 raw bytes 转换为其他字符串形式，并且再次过程中根据一个 bytes 创建一个 str

In [1]:
s = 'eggs'
s.encode()

b'eggs'

In [2]:
bytes(s,encoding='ascii')

b'eggs'

In [3]:
b = b'spam'

In [4]:
b.decode()

'spam'

In [5]:
str(b,encoding='ascii')

'spam'

不带编码的 str 调用返回 bytes 对象的打印字符串，而不是其 str 转换后的形式

In [6]:
import sys

In [7]:
sys.platform

'darwin'

In [8]:
sys.getdefaultencoding()

'utf-8'

In [9]:
bytes(s)

TypeError: string argument without an encoding

In [10]:
str(b)

"b'spam'"

In [11]:
len(str(b))

7

In [12]:
len(str(b,encoding='ascii'))

4

## 编码 Unicode 字符串

#### 编码 ASCII 文本

ASCII 文本是一种简单的 Unicode, 存储为表示字符的字节值的一个序列

In [13]:
ord('x')

120

In [14]:
chr(88)

'X'

In [15]:
s = 'XYZ'

In [16]:
s

'XYZ'

In [17]:
len(s)

3

In [18]:
[ord(c) for c in s]

[88, 89, 90]

对于常见的 7 为 ASCII 文本，按照 Unicode 编码方案，都是以每字节一个字符的方式表现的

In [19]:
s.encode('ascii')

b'XYZ'

In [20]:
s.encode('latin-1')

b'XYZ'

In [21]:
s.encode('utf-8')

b'XYZ'

以这种方式编码 ASCII 文本而返回 bytes 对象，本质上是一个较短整数序列，只不过这个序列可能打印为 ASCII 字符

In [22]:
s.encode('latin-1')[0]

88

In [23]:
list(s.encode('latin-1'))

[88, 89, 90]

#### 编码非 ASCII 文本

编码 非 ASCII 文本，需要在字符串中使用十六进制或 Unicode；十六进制转移限制于单个自己的值，但 Unicode 转义可以指定其值有两个和四个字节宽度的字符

In [24]:
chr(0xc4)

'Ä'

In [25]:
chr(0xc5)

'Å'

In [26]:
chr(0xe8)

'è'

In [27]:
s = '\xc4\xe8'

In [28]:
s

'Äè'

In [29]:
s = '\u00c4\u00e8'

In [30]:
s

'Äè'

In [31]:
len(s)

2

### 编码和解码非 ASCII 文本

如果试图把一个非 ASCII 字符串编码为 raw 像 ASCII 一样使用，会得到一个错误

In [32]:
s = '\u00c4\u00e8'

In [33]:
s

'Äè'

In [34]:
len(s)

2

In [35]:
s.encode('ascii')

UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-1: ordinal not in range(128)

In [36]:
s.encode('latin-1')

b'\xc4\xe8'

In [37]:
s.encode('utf-8')

b'\xc3\x84\xc3\xa8'

In [38]:
len(s.encode('latin-1'))

2

In [39]:
len(s.encode('utf-8'))

4

## 其他 Unicode 编码技术

支持为字符串制定 16 位或者 32 位的 Unicode 值

In [54]:
s = 'A\u00c4B\U0000000e8C'

In [55]:
s

'AÄB\x0e8C'

In [56]:
len(s)

6

In [57]:
s.encode('latin-1')

b'A\xc4B\x0e8C'

In [60]:
s.encode('utf-8')

b'A\xc3\x84B\x0e8C'

In [61]:
len(s.encode('utf-8'))

7

#### 转换编码

将字符串转换为不同于原字符集默认的一种编码，必须显式地提供一个编码名称以进行编码和解码

In [69]:
s = 'AABèC'

In [70]:
s

'AABèC'

In [71]:
s.encode()

b'AAB\xc3\xa8C'

In [72]:
t = s.encode('cp500')

In [73]:
t

b'\xc1\xc1\xc2T\xc3'

In [74]:
u = t.decode('cp500')

In [75]:
u

'AABèC'

## 源文件字符集编码声明

Python 默认使用 UTF-8 编码，但是允许我们通过包含一个注释来说明想要的编码

In [76]:
# -*- coding: latin-1 -*-

## 使用 Python 3 Bytes 对象

Python 3 bytes 对象支持的操作集

Python3 的 bytes 是一个较小的整数序列，其中每个整数都在 0 到 255 之间，并且在显示的时候恰好打印为 ASCII 字符

### 方法调用

通过下列方式来查看 str 拥有哪些 bytes 所没有的属性

In [77]:
set(dir('abc')) - set(dir(b'abc'))

{'casefold',
 'encode',
 'format',
 'format_map',
 'isdecimal',
 'isidentifier',
 'isnumeric',
 'isprintable'}

In [78]:
set(dir(b'abc')) - set(dir('abc'))

{'decode', 'fromhex', 'hex'}

常见的查找，替换，分片

In [79]:
b = b'spam'

In [80]:
b.find(b'pa')

1

In [81]:
b.replace(b'pa',b'xy')

b'sxym'

In [82]:
b.split(b'pa')

[b's', b'm']

In [83]:
b

b'spam'

In [84]:
b[0] = 'x'

TypeError: 'bytes' object does not support item assignment

序列操作

In [87]:
b = b'spam'

In [88]:
b

b'spam'

In [89]:
b[0]

115

In [90]:
b[-1]

109

In [92]:
chr(b[0])

's'

In [94]:
list(b)

[115, 112, 97, 109]

In [95]:
b + b'lmn'

b'spamlmn'

In [96]:
b * 4

b'spamspamspamspam'

创建 bytes 对象的其他方式

In [97]:
b = b'abc'

In [98]:
b

b'abc'

In [99]:
b = bytes('abc','ascii')

In [100]:
b

b'abc'

In [101]:
ord('a')

97

In [102]:
b = bytes([97,98,99])

In [103]:
b

b'abc'

str to bytes

In [104]:
b = 'spam'.encode()

In [108]:
b

b'spam'

bytes to str

In [106]:
s = b.decode()

In [107]:
s

'spam'

## 使用 Python3 的 bytearray 对象

In [ ]:
aa